# Evaluating Models on FS-Mol

`fs_mol.utils.test_utils.eval_model()` is a utility function that allows to evaluate a model against the full set of FS-Mol testing tasks, following a fixed evaluation scheme.

It requires the following arguments:
* `test_model_fn: Callable[[FSMolTaskSample, str, int], BinaryEvalMetrics]`:
  This is the core evaluation function, taking a `FSMolTaskSample` (that is, a sample of datapoints from a single task -- see notebooks/dataset.ipynb), a temporary output folder in which to store scratch data, and the seed used for the sampling.
  It should return `BinaryEvalMetric` object containing all metrics calculated from the model output and labels of the task sample.
* `dataset: FSMolDataset`:
  The actual dataset, which can be the full FS-Mol dataset as released, or a different set created directly through its constructor.
  This is particularly useful if you are planning to evaluate models on a set of tasks differing from FS-Mol's test set.

The returned results contain a list of evaluation metrics for each task, in a dictionary indexed by task name.

Additionally, it can be optionally further configured using the following optional arguments:
* `out_dir: Optional[str]`:
   If provided, a summary of the evaluation results will be written, as one CSV file per task.
* `seed: int`:
   A seed value used to make sampling and splitting of tasks deterministic.
   If set to anything but `0`, results of `eval_model` will be incomparable to the canonical evaluation runs.
* `num_samples: int`:
   The number of random splits to draw for the task undergoing evaluation.
* `train_set_sample_sizes: List[int]`:
   The sizes of training / support set data to consider. For each of these, `num_samples` samples will be drawn for each test task, and then be used to call `test_model_fn` with, where the passed `FSMolTaskSample` will have the requested number of `train_samples`.
* `valid_size_or_ratio: Union[int, float]`:
   Number or ratio of `train_samples` in the drawn samples that will be split out into `validation_samples`. This is useful for models that require an explicit validation set during fine-tuning.
* `test_size_or_ratio: Optional[Union[int, float, Tuple[int, int]]]`:
   Number or ratio of `test_samples` that will be drawn from the original task. By default, all available samples will be used, but it may be useful to limit this when not using `eval_model` for full model evaluation.
* `fold: DataFold`:
   The fold of FS-Mol on which to perform evaluation, typically will be the test fold.
* `task_reader_fn: Optional[Callable[[List[RichPath], int], Iterable[FSMolTask]]]`:
   Callable allowing additional transformations on the data prior to its batching and passing through a model.


## Example: Evaluating a Random Forest Model

In [27]:
# Setting up local details:
import os
import sys

# This should be the location of the checkout of the FS-Mol repository:
FS_MOL_CHECKOUT_PATH = os.path.join(os.environ['HOME'], "code", "ersilia", "ersilia-fsmol")
# FS_MOL_DATASET_PATH = os.path.join(os.environ['HOME'], "code", "ersilia", "ersilia-fsmol", "datasets", "fs-mol-data")
FS_MOL_DATASET_PATH = os.path.join(os.environ['HOME'], "code", "ersilia", "bioassays", "dataset")


os.chdir(FS_MOL_CHECKOUT_PATH)
sys.path.insert(0, FS_MOL_CHECKOUT_PATH)

We first define a simple implementation of a `test_model_fn` that creates a random forest model, using the scikit-learn default implementation and parameters.

In [28]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier

from fs_mol.data.fsmol_task import FSMolTaskSample
from fs_mol.utils.metrics import BinaryEvalMetrics, compute_binary_task_metrics

def test_model_fn(
    task_sample: FSMolTaskSample, temp_out_folder: str, seed: int
) -> BinaryEvalMetrics:
    train_data = task_sample.train_samples
    test_data = task_sample.test_samples

    # get data in to form for sklearn
    X_train = np.array([x.get_fingerprint() for x in train_data])
    X_test = np.array([x.get_fingerprint() for x in test_data])
    y_train = [float(x.bool_label) for x in train_data]
    y_test = [float(x.bool_label) for x in test_data]

    model = RandomForestClassifier()
    model.fit(X_train, y_train)

    # Compute test results:
    y_predicted_true_probs = model.predict_proba(X_test)[:, 1]
    test_metrics = compute_binary_task_metrics(y_predicted_true_probs, y_test)

    return test_metrics

Given these functions, we can now evaluate these models on the test tasks in FS-Mol as follows, reducing the space of considered samples from each task for speed purposes:

In [29]:
from fs_mol.data.fsmol_dataset import FSMolDataset, DataFold
from fs_mol.utils.test_utils import eval_model

fsmol_dataset = FSMolDataset.from_directory(FS_MOL_DATASET_PATH)

results = eval_model(
    test_model_fn=test_model_fn,
    dataset=fsmol_dataset,
    # Restrict number of samples to one per task:
    train_set_sample_sizes=[16],
    num_samples=1,
)

results

{'CHEMBL1000357': [FSMolTaskSampleEvalResults(size=11, acc=0.8181818181818182, balanced_acc=0.5, f1=0.9, prec=0.8181818181818182, recall=1.0, roc_auc=0.41666666666666663, avg_precision=0.8244187910854577, kappa=0.0, task_name='CHEMBL1000357', seed=0, num_train=16, num_test=11, fraction_pos_train=0.875, fraction_pos_test=0.8181818181818182)],
 'CHEMBL1006568': [FSMolTaskSampleEvalResults(size=67, acc=0.8059701492537313, balanced_acc=0.7829670329670331, f1=0.7346938775510204, prec=0.8571428571428571, recall=0.6428571428571429, roc_auc=0.9010989010989011, avg_precision=0.877757542991767, kappa=0.5866160417655435, task_name='CHEMBL1006568', seed=0, num_train=16, num_test=67, fraction_pos_train=0.4375, fraction_pos_test=0.417910447761194)],
 'CHEMBL1008279': [FSMolTaskSampleEvalResults(size=5, acc=0.6, balanced_acc=0.375, f1=0.0, prec=0.0, recall=0.0, roc_auc=0.5, avg_precision=0.3333333333333333, kappa=-0.25, task_name='CHEMBL1008279', seed=0, num_train=16, num_test=5, fraction_pos_train=0

In [30]:
import json

# Print out the number of results whose value is not None:
print(sum(1 for v in results.values() if any(x is not None for x in v)))

# Divide this by the total number of results:
print(sum(1 for v in results.values() if any(x is not None for x in v)) / len(results))

# Find a list of the keys whose value is not None:
# non_empty_keys = [k + ".jsonl.gz" for k in results.keys() if any(x is not None for x in results[k])]

# # Save to a json file of useful test data
# with open("test_filtered_thresh6.json", "w") as f:
# 	json.dump(non_empty_keys, f)

731
1.0
